<a href="https://colab.research.google.com/github/network-evolution/GenerativeAI_Tutorial_Notebooks/blob/main/02_04_chathistory_trimming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q langchain langchain_groq langgraph

In [ ]:
from google.colab import userdata
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, SystemMessage,AIMessage,trim_messages
from langchain_core.messages.utils import count_tokens_approximately
import os
os.environ['GROQ_API_KEY'] = userdata.get('groq')
model = ChatGroq(model="llama-3.3-70b-versatile", temperature=0)

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

#### define state Graph ####
workflow = StateGraph(state_schema=MessagesState)

### Create function to call LLM model ###
def call_model(state: MessagesState):
  system_prompt = (
        "You are a knowledgeable and helpful assistant. "
        "Answer all questions accurately and concisely. "
  )
  messages = [SystemMessage(content=system_prompt)] + state["messages"]

  # trimmed_message = trim_messages(messages=messages, token_counter=len,max_tokens=5, include_system=True)
  trimmed_message = trim_messages(messages=messages, token_counter=count_tokens_approximately, max_tokens=1000, include_system=True)

  print(f"Length of Trimmed message: {len(trimmed_message)}")
  print(f"Length of messages: {len(messages)}")

  print("Trimmed messages:")
  for msg in trimmed_message:
      if isinstance(msg, AIMessage):
          role = "AI"
      elif isinstance(msg, HumanMessage):
          role = "Human"
      elif isinstance(msg, SystemMessage):
          role = "System"
      else:
          role = "Unknown"
      print(f"{role}: {msg.content}\n")

  response = model.invoke(trimmed_message)
  return {"messages": response}

### Add node to Graph ######
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)


### add in-memory checkpointer ###
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
config = {"configurable": {"thread_id": "langgraph_test"}}
try:
  while True:
    print(f"\n \n{'_'*100}")
    user_input = input("Enter your question(or type 'exit'):")

    input_messages = [HumanMessage(content=user_input)]

    response = app.invoke({"messages": input_messages}, config=config)

    #### get token details ###
    print(f"{'*'*25}")
    token_data = response["messages"][-1].response_metadata.get("token_usage")
    input_token = token_data.get("prompt_tokens",0)
    output_token = token_data.get("completion_tokens",0)
    total_token = token_data.get("total_tokens",0)
    print(f"Input tokens: {input_token}")
    print(f"Output tokens: {output_token}")
    print(f"Total tokens: {total_token}")
    print(f"{'*'*25}")

    print(response)
    print(response["messages"][-1].content)

    if user_input.lower() == 'exit':
      print("Exiting...")
      break
except KeyboardInterrupt:
  print("Exiting...")


 
____________________________________________________________________________________________________
Enter your question(or type 'exit'):hi
Length of Trimmed message: 2
Length of messages: 2
Trimmed messages:
System: You are a knowledgeable and helpful assistant. Answer all questions accurately and concisely. 

Human: hi

*************************
Input tokens: 53
Output tokens: 10
Total tokens: 63
*************************
{'messages': [HumanMessage(content='hi', additional_kwargs={}, response_metadata={}, id='cfe55043-af28-41c9-bc45-73fdc6a95961'), AIMessage(content='Hello. How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 53, 'total_tokens': 63, 'completion_time': 0.036363636, 'prompt_time': 0.004240554, 'queue_time': 0.100466948, 'total_time': 0.04060419}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_6507bcfb6f', 'finish_reason': 'stop', 'logprobs': None}, id='run-a82047f9-6d76-4